<a href="https://colab.research.google.com/github/HuaichenOvO/ML/blob/main/CSC3180_prepreocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

致力于使用机器学习来为房价预测提供公正的、透明的模型。与中介无关，本模型用以帮助普通人。

本PJ的亮点在于它
技术上实现了xxx
结论上发现了xxx

<!-- bayesian thinking:
如果房价受A因素的波动显著地高于其他房子的平均水平，那么可以认为此时期内，A相关的 -->

In [76]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [77]:
import hashlib
import os
import tarfile
import zipfile
import requests



DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

def download(name, cache_dir=os.path.join('..', 'data')): 
    """下载一个DATA_HUB中的文件，返回本地文件名。"""
    assert name in DATA_HUB, f"{name} 不存在于 {DATA_HUB}."
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname  # Hit cache
    print(f'正在从{url}下载{fname}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname

def download_extract(name, folder=None):
    fname = download(name)
    base_dir = os.path.dirname(fname)
    data_dir, ext = os.path.splitext(fname)
    if ext == '.zip':
        fp = zipfile.ZipFile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        fp = tarfile.open(fname, 'r')
    else:
        assert False, '只有zip/tar文件可以被解压缩。'
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir

def download_all():
    """下载DATA_HUB中的所有文件。"""
    for name in DATA_HUB:
        download(name)

In [78]:
# !pip install d2l
# !pip install matplotlib==3.0.0
# !pip install --upgrade mxnet
# !pip install autogluon
# !pip install torch

In [161]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

DATA_HUB['kaggle_house_train'] = (  
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (  
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

In [ ]:
"""
数据可视化 了解大概印象
"""

# train_data.info()
# train_data.head(2)
# train_data['SalePrice']

# 热区图
corrmat = train_data.corr()
plt.subplots(figsize = (13,10))
sns.heatmap(corrmat)

In [166]:
"""
预处理
获得 panda dataframe 格式的训练集和测试集
"""
from sklearn.model_selection import train_test_split

# train_data['SalePrice'] = np.log(train_data['SalePrice'])

# train_data['SalePrice'] = train_data.apply(
#     lambda x:np.log(x['SalePrice']),axis=1)

# 将数据normalize成0~1，随后空白数据即可填上0
numeric_attri = train_data.dtypes[train_data.dtypes != 'object'].index
train_data[numeric_attri] = train_data[numeric_attri].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 将非numeric变量进行onehot处理
train_data = pd.get_dummies(train_data, dummy_na=True)
train_data[numeric_attri] = train_data[numeric_attri].fillna(0)
train_data.astype(float)

# split train_x and train_y
x = train_data.drop(['Id','SalePrice'], axis = 1)
y = train_data[['SalePrice']].astype(float)

print(y.head(10))

# there is no null data left
# print(y.isnull().sum().max())

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state= 42)

   SalePrice
0   0.347154
1   0.007286
2   0.535970
3  -0.515105
4   0.869545
5  -0.477341
6   1.587045
7   0.240159
8  -0.642241
9  -0.792034


In [ ]:
# 对 test data 进行同样的处理
test_data = pd.get_dummies(test_data, dummy_na=True)
numeric_attri = test_data.dtypes[test_data.dtypes != 'object'].index
test_data[numeric_attri] = test_data[numeric_attri].apply(
    lambda x: (x - x.mean()) / (x.std()))
test_data[numeric_attri] = test_data[numeric_attri].fillna(0)
test_data.astype(float)

In [82]:
from sklearn.metrics import mean_squared_error 

In [165]:
#Linear regression model - Linear regression model doesn't seem to be working well

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
reg = LinearRegression().fit(x_train, y_train)
y_pred = reg.predict(x_test)
RMSE_score = mean_squared_error(y_test, y_pred, squared = False)
print(RMSE_score)

Score  = 301

1627823803.0084786


In [123]:
# Model 3 - Catboost Regression 
from catboost import CatBoostRegressor

catb = CatBoostRegressor(iterations=3500,verbose=1000)
catb.fit(x_train, y_train)
y_pred = catb.predict(x_test)

RMSE_score = mean_squared_error(y_test, y_pred, squared = False)
print(RMSE_score)

Learning rate set to 0.015438
0:	learn: 0.9642050	total: 8.33ms	remaining: 29.1s
1000:	learn: 0.1605242	total: 6.34s	remaining: 15.8s
2000:	learn: 0.1017707	total: 12.7s	remaining: 9.52s
3000:	learn: 0.0698663	total: 19s	remaining: 3.16s
3499:	learn: 0.0592918	total: 22.1s	remaining: 0us
0.3495296097351661


In [124]:
# Model 4 - XGBoost Regression 
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(n_estimators=3000, learning_rate=0.005)
xgb_reg.fit(x_train, y_train, early_stopping_rounds=5, eval_set=[(x_test, y_test)],verbose = 100)
y_pred = xgb_reg.predict(x_test)

RMSE_score = mean_squared_error(y_test, y_pred, squared = False)
print(RMSE_score)

[0]	validation_0-rmse:1.28886
[100]	validation_0-rmse:0.89821
[200]	validation_0-rmse:0.65869
[300]	validation_0-rmse:0.51688
[400]	validation_0-rmse:0.44162
[500]	validation_0-rmse:0.39789
[600]	validation_0-rmse:0.37253
[700]	validation_0-rmse:0.35887
[800]	validation_0-rmse:0.35017
[900]	validation_0-rmse:0.34666
[1000]	validation_0-rmse:0.34389
[1100]	validation_0-rmse:0.34201
[1199]	validation_0-rmse:0.34077
0.3407254198014607
